## Tên thành viên:
* Lư Trung Hậu - 21127043
* Nguyễn Hồng Quế Anh - 21127578
* Lê Nguyễn Thái Dương - 21127733
* Trần Công Bảo - 21127737

### Named Entity Recognition (NER) with conll2003


- B-PER/I-PER means the word corresponds to the beginning of/is inside a person entity.
- B-ORG/I-ORG means the word corresponds to the beginning of/is inside an organization entity.
- B-LOC/I-LOC means the word corresponds to the beginning of/is inside a location entity.
- B-MISC/I-MISC means the word corresponds to the beginning of/is inside a miscellaneous entity.

In [1]:
# Install
!pip install transformers datasets tokenizers seqeval -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
import datasets
import numpy as np
from transformers import BertTokenizerFast
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification

conll2003 = datasets.load_dataset("conll2003", trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [3]:
conll2003

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [4]:
conll2003.shape

{'train': (14041, 5), 'validation': (3250, 5), 'test': (3453, 5)}

In [5]:
conll2003["train"].features["ner_tags"]

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [6]:
conll2003['train'].description

'The shared task of CoNLL-2003 concerns language-independent named entity recognition. We will concentrate on\nfour types of named entities: persons, locations, organizations and names of miscellaneous entities that do\nnot belong to the previous three groups.\n\nThe CoNLL-2003 shared task data files contain four columns separated by a single space. Each word has been put on\na separate line and there is an empty line after each sentence. The first item on each line is a word, the second\na part-of-speech (POS) tag, the third a syntactic chunk tag and the fourth the named entity tag. The chunk tags\nand the named entity tags have the format I-TYPE which means that the word is inside a phrase of type TYPE. Only\nif two phrases of the same type immediately follow each other, the first word of the second phrase will have tag\nB-TYPE to show that it starts a new phrase. A word with tag O is not part of a phrase. Note the dataset uses IOB2\ntagging scheme, whereas the original dataset uses 

In [7]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In thử

In [8]:
conll2003['train'][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

# số nhãn

In [9]:
example_text = conll2003['train'][0]

tokenized_input = tokenizer(example_text["tokens"], is_split_into_words=True)

tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])

word_ids = tokenized_input.word_ids()

print(word_ids)


[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, None]


In [10]:
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

['[CLS]',
 'eu',
 'rejects',
 'german',
 'call',
 'to',
 'boycott',
 'british',
 'lamb',
 '.',
 '[SEP]']

số lượng input_ids (danh sách các mã token mà tokenizer tạo ra) dài hơn so với danh sách các nhãn (labels) ban đầu của tập dữ liệu.

In [11]:
len(example_text['ner_tags']), len(tokenized_input["input_ids"])


(9, 11)

*  Gán nhãn -100 cho các token đặc biệt và các subwords muốn bỏ qua trong quá trình huấn luyện
* Bỏ qua (mask) các đại diện subword sau subword đầu tiên

In [12]:
def tokenize_and_align_labels(examples, label_all_tokens=True):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    aligned_labels = []  # Tạo danh sách
    # Duyệt
    for example_index, ner_labels in enumerate(examples["ner_tags"]):
        word_indices = tokenized_inputs.word_ids(batch_index=example_index)  # Lấy các chỉ số từ gốc trong câu

        previous_word_idx = None  # Lưu trữ chỉ số của từ trước đó để kiểm tra subword
        label_ids = []  # Danh sách lưu trữ các nhãn đã căn chỉnh cho câu hiện tại

        # Duyệt qua từng chỉ số từ trong câu đã được token hóa
        for word_idx in word_indices:
            if word_idx is None:
                # Nếu chỉ số từ là None, đó là các token đặc biệt như [CLS], [SEP] -> gán nhãn -100
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                # Nếu chỉ số từ hiện tại khác chỉ số từ trước đó, gán nhãn tương ứng cho từ đầu tiên
                label_ids.append(ner_labels[word_idx])
            else:
                label_ids.append(ner_labels[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx
        aligned_labels.append(label_ids)

    tokenized_inputs["labels"] = aligned_labels
    return tokenized_inputs  # Trả về các token đã được token hóa


In [13]:
conll2003['train'][5:6]

{'id': ['5'],
 'tokens': [['"',
   'We',
   'do',
   "n't",
   'support',
   'any',
   'such',
   'recommendation',
   'because',
   'we',
   'do',
   "n't",
   'see',
   'any',
   'grounds',
   'for',
   'it',
   ',',
   '"',
   'the',
   'Commission',
   "'s",
   'chief',
   'spokesman',
   'Nikolaus',
   'van',
   'der',
   'Pas',
   'told',
   'a',
   'news',
   'briefing',
   '.']],
 'pos_tags': [[0,
   28,
   41,
   30,
   37,
   12,
   16,
   21,
   15,
   28,
   41,
   30,
   37,
   12,
   24,
   15,
   28,
   6,
   0,
   12,
   22,
   27,
   16,
   21,
   22,
   22,
   14,
   22,
   38,
   12,
   21,
   21,
   7]],
 'chunk_tags': [[0,
   11,
   21,
   22,
   22,
   11,
   12,
   12,
   17,
   11,
   21,
   22,
   22,
   11,
   12,
   13,
   11,
   0,
   0,
   11,
   12,
   11,
   12,
   12,
   12,
   12,
   12,
   12,
   21,
   11,
   12,
   12,
   0]],
 'ner_tags': [[0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,

In [14]:
q = tokenize_and_align_labels(conll2003['train'][4:5])
print(q)

{'input_ids': [[101, 2762, 1005, 1055, 4387, 2000, 1996, 2647, 2586, 1005, 1055, 15651, 2837, 14121, 1062, 9328, 5804, 2056, 2006, 9317, 10390, 2323, 4965, 8351, 4168, 4017, 2013, 3032, 2060, 2084, 3725, 2127, 1996, 4045, 6040, 2001, 24509, 1012, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 5, 0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, -100]]}


sau khi áp dụng tokenize_and_align_labels, dữ liệu không chỉ bao gồm các mã token (input_ids), loại token (token_type_ids), và mặt nạ chú ý (attention_mask), mà còn bao gồm cả các nhãn đã được căn chỉnh với từng token (labels)   mục đích là để phân định các nahnx khác nhau

In [15]:
# Lặp qua từng token và nhãn tương ứng, sau đó in ra một cách rõ ràng và có định dạng
for token, label in zip(tokenizer.convert_ids_to_tokens(q["input_ids"][0]), q["labels"][0]):
    label_str = f"{label}" if label != -100 else "IGNORED"  # Hiển thị "IGNORED" nếu label là -100
    print(f"{token:<15} -> {label_str}")


[CLS]           -> IGNORED
germany         -> 5
'               -> 0
s               -> 0
representative  -> 0
to              -> 0
the             -> 0
european        -> 3
union           -> 4
'               -> 0
s               -> 0
veterinary      -> 0
committee       -> 0
werner          -> 1
z               -> 2
##wing          -> 2
##mann          -> 2
said            -> 0
on              -> 0
wednesday       -> 0
consumers       -> 0
should          -> 0
buy             -> 0
sheep           -> 0
##me            -> 0
##at            -> 0
from            -> 0
countries       -> 0
other           -> 0
than            -> 0
britain         -> 5
until           -> 0
the             -> 0
scientific      -> 0
advice          -> 0
was             -> 0
clearer         -> 0
.               -> 0
[SEP]           -> IGNORED


In [16]:
## Applying on entire data
tokenized_datasets = conll2003.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [17]:
tokenized_datasets['train'][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0],
 'input_ids': [101,
  7327,
  19164,
  2446,
  2655,
  2000,
  17757,
  2329,
  12559,
  1012,
  102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, -100]}

In [18]:
# Defining model

model = AutoModelForTokenClassification.from_pretrained("bert-base-uncased", num_labels=9)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
#Define training args
from transformers import TrainingArguments, Trainer


args = TrainingArguments(
"test-ner",
evaluation_strategy = "epoch",
learning_rate=2e-5,
per_device_train_batch_size=16,
per_device_eval_batch_size=16,
num_train_epochs=10,
weight_decay=0.01,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [20]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [21]:
metric = datasets.load_metric("seqeval", trust_remote_code=True)

<ipython-input-21-0589bf74b14b>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric("seqeval", trust_remote_code=True)


### Lets test the metrix on an example

In [22]:
example = conll2003['train'][0]

In [23]:
label_list = conll2003["train"].features["ner_tags"].feature.names

label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [24]:
for i in example["ner_tags"]:
  print(i)

3
0
7
0
0
0
7
0
0


In [25]:
labels = [label_list[i] for i in example["ner_tags"]]
labels

['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

In [26]:
metric.compute(predictions=[labels], references=[labels])

{'MISC': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

### Compute Metrics


In [27]:
def compute_metrics(eval_predictions):
    # Tách các logits dự đoán và nhãn thực tế từ kết quả đánh giá
    logits, true_labels = eval_predictions

    # Lấy nhãn dự đoán bằng cách chọn nhãn có xác suất cao nhất
    predicted_labels = np.argmax(logits, axis=2)

    # Loại bỏ các giá trị với nhãn thực tế là -100, tức là các token đặc biệt hoặc subwords không tham gia huấn luyện
    filtered_predictions = [
        [label_list[pred] for (pred, label) in zip(prediction, labels) if label != -100]
        for prediction, labels in zip(predicted_labels, true_labels)
    ]

    filtered_true_labels = [
        [label_list[label] for (pred, label) in zip(prediction, labels) if label != -100]
        for prediction, labels in zip(predicted_labels, true_labels)
    ]

    # Tính toán các chỉ số Precision, Recall, F1-score và Accuracy
    metrics_result = metric.compute(predictions=filtered_predictions, references=filtered_true_labels)

    # Trả về kết quả với các chỉ số được tách riêng
    return {
        "precision": metrics_result["overall_precision"],
        "recall": metrics_result["overall_recall"],
        "f1": metrics_result["overall_f1"],
        "accuracy": metrics_result["overall_accuracy"],
    }


## Training

In [28]:
trainer = Trainer(
   model,
   args,
   train_dataset=tokenized_datasets["train"],
   eval_dataset=tokenized_datasets["validation"],
   data_collator=data_collator,
   tokenizer=tokenizer,
   compute_metrics=compute_metrics
)

In [29]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.218100,0.067077,0.917556,0.923817,0.920676,0.982096
2,0.046800,0.055899,0.934746,0.942275,0.938496,0.985162
3,0.026000,0.056130,0.942039,0.949099,0.945556,0.986687
4,0.015500,0.062327,0.935537,0.949771,0.942600,0.986147
5,0.010300,0.067879,0.935292,0.947533,0.941373,0.985925
6,0.005800,0.068402,0.939508,0.948652,0.944058,0.986290
7,0.005100,0.069537,0.941952,0.951225,0.946566,0.986878
8,0.003700,0.074450,0.941587,0.950330,0.945938,0.986878
9,0.002900,0.077461,0.944407,0.952120,0.948248,0.987227
10,0.001900,0.077178,0.941827,0.952679,0.947222,0.987132


TrainOutput(global_step=8780, training_loss=0.027377243031130292, metrics={'train_runtime': 1756.2968, 'train_samples_per_second': 79.947, 'train_steps_per_second': 4.999, 'total_flos': 3405845692350414.0, 'train_loss': 0.027377243031130292, 'epoch': 10.0})



# Save Model

In [30]:
## Save model
model.save_pretrained("ner_model")

In [31]:
## Save tokenizer
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/vocab.txt',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [32]:
id2label = {
    str(i): label for i,label in enumerate(label_list)
}
label2id = {
    label: str(i) for i,label in enumerate(label_list)
}

## Loading model & prediction

In [33]:
import json

In [34]:
config = json.load(open("ner_model/config.json"))
config["id2label"] = id2label
config["label2id"] = label2id
json.dump(config, open("ner_model/config.json","w"))

In [35]:
model_fine_tuned = AutoModelForTokenClassification.from_pretrained("ner_model")

In [36]:
from transformers import pipeline

In [38]:
nlp = pipeline("ner", model=model_fine_tuned, tokenizer=tokenizer)


example = "Joe Biden is a Democrat, and the current president of the United States. Microsoft is one of the biggest company in the world."

ner_results = nlp(example)

print(ner_results)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'entity': 'B-PER', 'score': 0.9997428, 'index': 1, 'word': 'joe', 'start': 0, 'end': 3}, {'entity': 'I-PER', 'score': 0.99973756, 'index': 2, 'word': 'bid', 'start': 4, 'end': 7}, {'entity': 'I-PER', 'score': 0.9997948, 'index': 3, 'word': '##en', 'start': 7, 'end': 9}, {'entity': 'B-MISC', 'score': 0.9994893, 'index': 6, 'word': 'democrat', 'start': 15, 'end': 23}, {'entity': 'B-LOC', 'score': 0.99975246, 'index': 14, 'word': 'united', 'start': 58, 'end': 64}, {'entity': 'I-LOC', 'score': 0.99959546, 'index': 15, 'word': 'states', 'start': 65, 'end': 71}, {'entity': 'B-ORG', 'score': 0.9993729, 'index': 17, 'word': 'microsoft', 'start': 73, 'end': 82}]
